# Restaurar Planilla de Datos Geográficos

Esta permite subir, validar y restaurar la planilla de datos geográficos, que la aplicación de licencias y registros usará de manera preferencial.

El archivo deber ser de formato "CSV" y contar con las siguientes columnas:

* **nombre**: Nombre de prueba (str)
* **x**: Coordenada X (float) 
* **y**: Coordenada Y (float)
* **utm**: Zona UTM (str)

De ser valido el archivo reemplazará al anterior y será usado por la aplicación de manera preferencial. Si la aplicación no encuentra los datos geográficos que busca en este archivo, hará uso de los datos obtenidos desde el SIIRAyS.

## Instrucciones

La tabla muestra el archivo usado actualmente por la aplicación.

Al presionar el botón **"Upload (0)"** y seleccionar el nuevo archivo, este es validado. Si el archivo pasa el proceso de validación, se mostrarán algunos detalles del archivo y la tabla será actualizada, pero todavía no reemplazará a la versión previa.

De estar de acuerdo con los datos mostrados, al hacer click en **"Reemplazar"**, el archivo usado por las aplicaciones será reemplazado.

---

In [ ]:
import ipywidgets as widgets
import markdown
import qgrid
import numpy as np
import pandas as pd
import pandas_schema as ps
from pandas_schema.validation import InListValidation, IsDtypeValidation
import json
from io import BytesIO

import os
from os.path import join, exists

file_name = "geo_data.csv"
file_path = join(os.getcwd(),"datasets",file_name)
DATA_EXISTS = exists(file_path)

grid_options = {
    # SlickGrid options
    'fullWidthRows': True,
    'syncColumnCellResize': True,
    'forceFitColumns': False,
    'defaultColumnWidth': 150,
    'rowHeight': 28,
    'enableColumnReorder': False,
    'enableTextSelectionOnCells': True,
    'editable': True,
    'autoEdit': False,
    'explicitInitialization': True,

    # Qgrid options
    'maxVisibleRows': 15,
    'minVisibleRows': 8,
    'sortable': True,
    'filterable': True,
    'highlightSelectedCell': False,
    'highlightSelectedRow': True
}

schema = ps.Schema([
    ps.Column(
        name = "nombre",
        validations = [
            IsDtypeValidation(dtype=np.object),
        ],
        allow_empty = False,
    ),
    ps.Column(
        name = "x",
        validations = [
            IsDtypeValidation(dtype=np.floating),
        ],
        allow_empty = False,
    ),
    ps.Column(
        name = "y",
        validations = [
            IsDtypeValidation(dtype=np.floating),
        ],
        allow_empty = False,
    ),
    ps.Column(
        name = "utm",
        validations = [
            IsDtypeValidation(dtype=np.object),
            InListValidation(
                options = ["19K", "20K"],
                case_sensitive = False,
            )
        ],
        allow_empty = False,
    ),
])

In [ ]:
upload_widget = widgets.FileUpload(
    accept = ".csv",
    multiple = False,
    button_style = "info",
)
                         
help_html = widgets.HTML(value = "")
                         
grid = qgrid.QGridWidget(
    df = pd.read_csv(file_path) if DATA_EXISTS else pd.DataFrame(),
    grid_options = grid_options,
)

restore_button = widgets.Button(
    description = "Reemplazar",
    disabled = True,
    button_style = "success",
    tooltip = 'Reemplazar la planilla de datos para la aplicación.',
    icon = 'archive',
)
restore_html = widgets.HTML()

def on_upload_widget_value_change(change):
    # Limpeza de widgets
    grid.df = pd.DataFrame()
    restore_html.value = ""
    
    # Recuperación de datos del widgets de archivo
    file_val = list(change["new"].values())[0]
    metadata = file_val["metadata"]
    content = file_val["content"]
    
    # Validación: "Archivo es CSV"
    if metadata["name"][-4:] != ".csv":
        help_html.value = "El archivo debe ser del tipo CSV"
        return
    
    # Lectura del archivo
    input_df = pd.read_csv(BytesIO(content))
    
    # Validación: "Planilla tiene el esquema correcto"
    errors = schema.validate(input_df)
    if len(errors) > 0:
        error_msgs = "<br>".join([str(e) for e in errors])
        help_html.value = f"El archivo ingresado no es válido. Errores:<br>{error_msgs}"
        return
    
    # Archivo Válido. Mostrar detalles
    try:
        details = "<br>".join([f"{k}: {v}" for k,v in metadata.items()])
        help_html.value = f"El archivo ingresado es válido. Detalles:<br>{details}"
        grid.df = input_df
        restore_button.disabled = False
    except Exception as e:
        print(e)
    
def on_observe_button_click(b):
    grid.df.to_csv(file_path)
    restore_html.value = "Archivo reemplazado."
    

upload_widget.observe(on_upload_widget_value_change, names="value")
restore_button.on_click(on_observe_button_click)

app = widgets.VBox([
    upload_widget,
    help_html,
    grid,
    widgets.HBox([restore_button, restore_html]),
])

In [ ]:
app